*ANALISIS DE TEMPERATURA SUPERFICIAL A TRAVÉS DE MODIS*

## Descargando paquetes necesarios

In [1]:
import ee
import os
import geemap
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Accediendo a los datos de temperatura del sensor MODIS a 1 km

In [3]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
AOI = ee.FeatureCollection('users/Guillermo-Ortega/Limites/Municipios')
#AOI = ee.FeatureCollection('users/Guillermo-Ortega/Muestra_Roger/parcela_muestral')
def imagen (img):
        LST_Media_1km = img.select(['LST_Day_1km']) \
        .add(img.select(['LST_Night_1km'])) \
        .divide(2).rename('LST_Media_1km')
        return img.addBands(LST_Media_1km)

Temperatura = ee.ImageCollection('MODIS/061/MOD11A2') \
                .filterBounds(AOI) \
                .filter(ee.Filter.calendarRange(2022, 2022, 'year')) \
                .filter(ee.Filter.calendarRange(143,149, 'day_of_year')) \
                .map(imagen) \
                .select('LST_Night_1km', 'LST_Media_1km', 'LST_Day_1km') \
                .median() \
                .multiply(0.02) \
                .subtract(273.15) \
                .clip(AOI)

# direccion = os.path.expanduser('~/Desktop/Clima/TELEDETECCIÓN 2022/Clima/T_MODIS_2001-2022')
# salida = os.path.join(direccion, 'T_131_140_2022.csv')
# if not os.path.exists(direccion):
#        os.makedirs(direccion)
# geemap.zonal_statistics(Temperatura, AOI, salida, statistics_type = 'MEDIAN', scale = 1000)

temperatureVis = {'min': 17, 'max': 38, 'palette':
                ['040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
                 '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
                 '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
                 'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
                 'ff0000', 'de0101', 'c21301', 'a71001', '911003']
                 }

Map.addLayer(Temperatura.select('LST_Media_1km'), temperatureVis, 'temperatura')
#Map.addLayer(AOI, {}, 'Parcela')
Map.centerObject(AOI, 14)
Map

Map(center=[12.927244498927001, -85.01289157265876], controls=(WidgetControl(options=['position', 'transparent…

## Añadiendo leyenda 

In [ ]:
colors = temperatureVis['palette']
vmin = temperatureVis['min']
vmax = temperatureVis['max']
Map.add_colorbar(temperatureVis, label = 'Temperatura media °C', orientation = 'vertical')

## Analizando variabilidad de T° por departamento

In [ ]:
ruta_Tem = "C:/Users/guillermo.ortega/Desktop/TELEDETECCIÓN 2022/Clima/T_MODIS_2001-2022/T_MODIS 2001 - 2022.xlsx"
temperatura = pd.read_excel(ruta_Tem, sheet_name = "2022").sort_values("DEPTO")
temperatura['T_Max_14'].describe()

In [ ]:
f, ax = plt.subplots(figsize = (10, 5))
sns.boxplot(x = "DEPTO", y = "T_Min_14", data = temperatura).set(
    xlabel = "Departamentos", ylabel = "Temperatura °C", 
    title = "Registros de temperatura nocturna durante el periodo del 10 al 20 de mayo del 2022")
plt.xticks(rotation = 285)
plt.show()

## Analizando variabilidad de T° a través del tiempo

In [ ]:
temepartura_1 = pd.read_excel(ruta_Tem, sheet_name = '2001 - 2022').sort_values('DEPTO')
#temepartura_1.head()

In [ ]:
f, ax = plt.subplots(figsize = (12, 5))
sns.boxplot(x = 'Anio', y = 'T_Min_14', data = temepartura_1).set(
    title = 'Comportamiento de las temperaturas, periodo 10 al 20 de mayo, 2001 - 2022',
    xlabel = 'Años', ylabel = 'Temperatura en °C')
plt.xticks(rotation = 285)
plt.show()